In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pymc3 as pm
import arviz as az
import pandas as pd

In [2]:
data = pd.read_csv("fish.csv")
x = data['Width']
y = data['Height']

In [6]:
# Step 2: Define the Linear Model using PyMC3
with pm.Model() as linear_model:
    alpha_linear = pm.Normal('alpha_linear', mu=0, sd=10)
    beta_linear = pm.Normal('beta_linear', mu=0, sd=10)
    epsilon_linear = pm.HalfNormal('epsilon_linear', sd=1)
    mu_linear = alpha_linear + beta_linear * x
    likelihood_linear = pm.Normal('likelihood_linear', mu=mu_linear, sd=epsilon_linear, observed=y)

# Step 3: Define the Polynomial^2 Model using PyMC3
with pm.Model() as poly_model:
    alpha_poly = pm.Normal('alpha_poly', mu=0, sd=10)
    beta1_poly = pm.Normal('beta1_poly', mu=0, sd=10)
    beta2_poly = pm.Normal('beta2_poly', mu=0, sd=10)
    epsilon_poly = pm.HalfNormal('epsilon_poly', sd=1)
    mu_poly = alpha_poly + beta1_poly * x + beta2_poly * x**2
    likelihood_poly = pm.Normal('likelihood_poly', mu=mu_poly, sd=epsilon_poly, observed=y)

# Step 4: Fit the models
with linear_model:
    linear_trace = pm.sample(2000, tune=1000)

with poly_model:
    poly_trace = pm.sample(2000, tune=1000)

# Step 5: Calculate LOO or WAIC
linear_waic = az.waic(linear_trace, linear_model)
poly_waic = az.waic(poly_trace, poly_model)

# Step 6: Extract relevant information
elpd_loo_waic = [linear_waic.waic, poly_waic.waic]
p_loo_waic = [linear_waic.p_waic, poly_waic.p_waic]
elpd_diff = linear_waic.waic - poly_waic.waic

# Step 7: Create the DataFrame
df = pd.DataFrame({
    'Model': ['Linear', 'Polynomial^2'],
    'elpd_loo_waic': elpd_loo_waic,
    'p_loo_waic': p_loo_waic,
    'elpd_diff': elpd_diff
})

# Display the resulting DataFrame
print(df)


/Applications/anaconda3/envs/dice/lib/python3.8/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [epsilon_linear, beta_linear, alpha_linear]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 10 seconds.
/Applications/anaconda3/envs/dice/lib/python3.8/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [epsilon_poly, beta2_poly, beta1_poly, alpha_poly]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 11 seconds.
The number of effective samples is smaller than 25% for some parameters.
Got error No model on context stack. trying to find log_likelihood in translation.
/Applications/anaconda3/envs/dice/lib/python3.8/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(
Got error No model on context stack. trying to find log_likelihood in translation.
/Applications/anaconda3/envs/dice/lib/python3.8/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_

          Model  elpd_loo_waic  p_loo_waic  elpd_diff
0        Linear    -380.677012    2.705710  -4.405435
1  Polynomial^2    -376.271577    3.091168  -4.405435


In [7]:
# Provided information
linear_elpd_loo_waic_modified = -380.606466
linear_p_loo_waic_modified = 2.667605
poly_elpd_loo_waic_modified = -376.308750
poly_p_loo_waic_modified = 3.123392
elpd_diff_modified = -4.297716

print ("\nQuestion 1\n")

# Print the summary
print("The elpd_loo_waic represents the estimated log pointwise predictive density under leave-one-out cross-validation.")
print("The higher the value, the better the model.")

print("\nLinear Model:")
print(f" - elpd_loo_waic: {linear_elpd_loo_waic_modified}")
print(f" - p_loo_waic: {linear_p_loo_waic_modified}")

print("\nPolynomial^2 Model:")
print(f" - elpd_loo_waic: {poly_elpd_loo_waic_modified}")
print(f" - p_loo_waic: {poly_p_loo_waic_modified}")

print("\nThe p_loo_waic is the effective number of parameters.")
print("Lower values are generally better, as they suggest a more parsimonious model.")

print("\nIn this case:")
print(f" - Linear model has a lower p_loo_waic value: {linear_p_loo_waic_modified}")
print(f" - Polynomial^2 model has a higher p_loo_waic value: {poly_p_loo_waic_modified}")

print("\nThe elpd_diff is the difference in expected log predictive density between models.")
print("It gives an indication of how much better one model is than the other.")

print("\nIn this case:")
print(f" - elpd_diff is negative, indicating that the Linear model is slightly worse than the Polynomial^2 model: {elpd_diff_modified}")



Question 1

The elpd_loo_waic represents the estimated log pointwise predictive density under leave-one-out cross-validation.
The higher the value, the better the model.

Linear Model:
 - elpd_loo_waic: -380.606466
 - p_loo_waic: 2.667605

Polynomial^2 Model:
 - elpd_loo_waic: -376.30875
 - p_loo_waic: 3.123392

The p_loo_waic is the effective number of parameters.
Lower values are generally better, as they suggest a more parsimonious model.

In this case:
 - Linear model has a lower p_loo_waic value: 2.667605
 - Polynomial^2 model has a higher p_loo_waic value: 3.123392

The elpd_diff is the difference in expected log predictive density between models.
It gives an indication of how much better one model is than the other.

In this case:
 - elpd_diff is negative, indicating that the Linear model is slightly worse than the Polynomial^2 model: -4.297716


In [8]:
# Provided information
linear_p_loo_waic_modified = 2.667605
poly_p_loo_waic_modified = 3.123392

print ("\nQuestion 2\n")

# Print the summary
print("Linear Model:")
print(f"\n - p_loo_waic: {linear_p_loo_waic_modified}")

print("\nPolynomial^2 Model:")
print(f"\n - p_loo_waic: {poly_p_loo_waic_modified}")

print("\nThe p_loo_waic for the Polynomial^2 model is higher than that of the Linear model.")
print("Therefore, based on the table, the Polynomial^2 model is considered more complex.")
print("This is likely due to the additional parameters introduced by the quadratic term (beta2 * x^2),")
print("making the Polynomial^2 model more flexible and potentially able to capture more complex patterns in the data.")

print("\nIt's important to note that the choice between a simpler and more complex model often involves a trade-off")
print("between model fit and model simplicity. In this case, the decision on which model to choose may depend on")
print("the specific goals of your analysis and your willingness to trade off complexity for predictive accuracy.")



Question 2

Linear Model:

 - p_loo_waic: 2.667605

Polynomial^2 Model:

 - p_loo_waic: 3.123392

The p_loo_waic for the Polynomial^2 model is higher than that of the Linear model.
Therefore, based on the table, the Polynomial^2 model is considered more complex.
This is likely due to the additional parameters introduced by the quadratic term (beta2 * x^2),
making the Polynomial^2 model more flexible and potentially able to capture more complex patterns in the data.

It's important to note that the choice between a simpler and more complex model often involves a trade-off
between model fit and model simplicity. In this case, the decision on which model to choose may depend on
the specific goals of your analysis and your willingness to trade off complexity for predictive accuracy.
